In [2]:
from datetime import datetime, timedelta

import pandas as pd
import glob, os
import pickle
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec
from matplotlib.dates import DateFormatter
%matplotlib inline
# for applying kfold validation set
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
# for logistic regression
from sklearn.linear_model import LogisticRegression
# convert date into float
import datetime as dt 
# converting target to labels
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from random import sample

In [ ]:
# print the channel list
#channel_list

In [ ]:
# for the purpose of testing
# setting the channel list 
#channel_list = [103222, 9000, 55227,54454, 51181]
channel_list = [56269, 9000, 50959, 55227, 54454, 51181, 103222]
#channel_list = [51181, 54454]
#channel_list = [9000]
# good results with 10 fold: 55227, 9000, 54454, 51181

# error with 56269, 50959: RESOLVED
# This solver needs samples of at least 2 classes in the data, 
# but the data contains only one class: 0. 

# error with 103222: RESOLVED
# Cannot have number of splits n_splits=10 greater than the number of samples: 0.

In [3]:
# Read layer3-60 data from all the csv files 
path = r'C:/Users/jainsac/Downloads/MRP/Assignment 2/Data'
data = pd.concat(map(lambda file: pd.read_csv(file, header = None), glob.glob(os.path.join('', path+'/*.csv'))))

In [6]:
data.head(3)

,StartTime,EndTime,Channel,Power_dbm,SNR,Occupancy
0,2016-10-06 00:00:00.1640000,2016-10-06 01:00:00.0000000,56302,-77.940605,26.758839,100.0
1,2016-10-06 00:00:00.1640000,2016-10-06 01:00:00.0000000,56303,-83.293709,21.397917,100.0
2,2016-10-06 00:00:00.1640000,2016-10-06 01:00:00.0000000,56304,-63.193748,41.497952,100.0


In [5]:
data.columns = ['StartTime', 'EndTime', 'Channel', 'Power_dbm','SNR', 'Occupancy']

In [ ]:
# plt.hist(data['Occupancy'])
# plt.xlabel("Occupancy")
# plt.ylabel("Frequency")
# plt.title("Histogram for Occupancy attribute")

In [ ]:
# plt.hist(data['SNR'])
# plt.xlabel("SNR")
# plt.ylabel("Frequency")
# plt.title("Histogram for SNR attribute")

In [ ]:
# plt.hist(data['Power_dbm'])
# plt.xlabel("Power_dbm")
# plt.ylabel("Frequency")
# plt.title("Histogram for Power_dbm attribute")

In [ ]:
# import seaborn as sns
# corr = data.corr()
# sns.heatmap(corr, 
#             xticklabels=corr.columns.values,
#             yticklabels=corr.columns.values)

In [ ]:
# data['Power_dbm'].corr(data['SNR'])

In [ ]:
# Copying the df to a new dataframe so that 
# future processing could be done easily
Temporaldf = data

In [ ]:
channel_list = data.Channel.unique().tolist()

In [ ]:
#channel_list = [54048, 56302, 9000, 50959, 55227, 54454, 51181]
#channel_list = [9000]
# for the Temporaldf take start time and sort it and 

uniqueST = Temporaldf.StartTime.unique()
ST_df=pd.DataFrame(uniqueST, columns=['StartT']) 
TemporalDFInd = ST_df.set_index('StartT')
TempReindex = TemporalDFInd.reindex(pd.date_range(start=TemporalDFInd.index[0], end=TemporalDFInd.index[-1], freq='1h'))
reindexed = TempReindex.shape[0]

channelO = []
for ch in channel_list:
    TemporalDF = Temporaldf[(Temporaldf.Channel == ch)][['StartTime']]
    #print(TemporalDF)
    #TemporalDFInd = TemporalDF.set_index('StartTime')
    original = TemporalDF.shape[0]
    #print(original)
    #TempReindex = TemporalDFInd.reindex(pd.date_range(start=TemporalDFInd.index[0], end=TemporalDFInd.index[-1], freq='1h'))
    #print(TempReindex)
    #reindexed = 624#TempReindex.shape[0]
    #print(reindexed)
    channelOcc = (original/reindexed)*100
    #print(channelOcc)
    channelO.append(channelOcc)
    # Put channel and channelOcc in DF
temp_df = pd.DataFrame({'Channel': channel_list, 'perData': channelO})

In [ ]:
#plt.hist(temp_df['perData'])
#plt.xlabel("% Data Available")
#plt.ylabel("Channel Frequency")
#plt.title("Percent data available for the month of October")

In [ ]:
dataThresh = np.percentile(channelO, 75)

temp_df = temp_df[temp_df['perData'] >= dataThresh]
channel_list = temp_df['Channel'].tolist()

In [ ]:
merged = pd.merge(temp_df,data, on=['Channel','Channel'])

In [ ]:
merged = merged.drop('perData', 1)

In [ ]:
# Dropping the data point if null value is encountered in 
# Occupancy, Power_dbm or SNR. how = 'any' is the default
# option which means if any of them is NA, drop the entire
# row of the data from the dataset. 
merged = merged.dropna(subset=['Occupancy', 'Power_dbm', 'SNR'], how='any')

In [ ]:
TimeInducedDfOriginal = merged

In [ ]:
# Converting the string StartTime to unique float value
merged['StartTime'] = pd.to_datetime(merged['StartTime'])
merged['StartTime'] = merged['StartTime'].apply(lambda v: v.timestamp())

# Converting the string EndTime to unique float value
merged['EndTime'] = pd.to_datetime(merged['EndTime']) 
merged['EndTime'] = merged['EndTime'].apply(lambda v: v.timestamp())

# Convert the occupancy into integer value
merged['Occupancy'] = merged['Occupancy'].astype(np.int64)

#### data['OccupancyClass'] 

In [ ]:
# plt.hist(merged['Occupancy'])

In [ ]:
# This will label the dataframe on Occupancy and save this new column in the dataframe. 
# Occupancy percent < 50 will be classified as 0
# Occupancy percent > 50 will be classified as 1 
merged['OccupancyClass'] = pd.cut(merged['Occupancy'], [-np.inf, 50, np.inf], labels=[0, 1])

In [ ]:
merged.head()

In [ ]:
from sklearn.preprocessing import Imputer, LabelEncoder, OneHotEncoder, StandardScaler

# number of splits in K-fold
numOfSplits = 10

# iterating for each channel in channel list
for ch in channel_list:
    print('CHANNEL PICKED FOR PREDICTION:{:.0f}'.format(ch))
    print("********************************************************")
    
    # split the dataframe into two sets: x - attributes, y - response variables
    # in our case, y = Occupancy, x would be rest of the columns
    layer3dataPerChannelx = merged[(merged.Channel == ch)][['StartTime', 'EndTime', 'Channel', 'Power_dbm','SNR']]
    layer3dataPerChannely = merged[(merged.Channel == ch)][['OccupancyClass']]  
    
        
    # initialize variables for each channel
    n = len(layer3dataPerChannelx)
    kf = KFold(n_splits = numOfSplits)
    fold = 0
    LRaccuracy = []
    NBaccuracy = []
    LRFullaccuracy = []
    NBFullaccuracy = []
    FoldCount = []
    
    # IF condition is required to make an exception for the channels
    # where enough data is not available to create numOfSplits folds
    if n > numOfSplits: 
        for train_idx, test_idx in kf.split(layer3dataPerChannelx):
            #print('Train: %s | test: %s' % (train_idx, test_idx))
            print('Train length: %s | test length: %s' % (len(train_idx), len(test_idx))) #464, 52
            X_train, X_test = layer3dataPerChannelx.iloc[train_idx], layer3dataPerChannelx.iloc[test_idx]
            y_train, y_test = layer3dataPerChannely.iloc[train_idx], layer3dataPerChannely.iloc[test_idx]

           
            X_trainNew, X_testNew = layer3dataPerChannelx.iloc[train_idx, 0:4], layer3dataPerChannelx.iloc[test_idx, 0:4]
            y_trainNew, y_testNew = layer3dataPerChannely.iloc[train_idx], layer3dataPerChannely.iloc[test_idx]
            
            # check if values in y_train are more than 1. 
            if len(y_train.OccupancyClass.unique()) > 1:
                
                fold += 1
                FoldCount.append(fold)

                # Logistic Regression
                clf = LogisticRegression().fit(X_train, y_train)
                y_pred = clf.predict(X_test)
                
                scoreLR = clf.score(X_test, y_test)
                print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(scoreLR))
                LRFullaccuracy.append(scoreLR)
                

                # Confusion Matrix
                cfm_LR = confusion_matrix(y_test, y_pred)
                print("********************************************************")
                print("******* CONFUSION MATRIX FOR LOGISTIC REGRESSION *******")
                print("********************************************************")
                print(cfm_LR)

                #Create a Naive Bayes Classifier
                model = GaussianNB()
                # Train the model using the training sets 
                model.fit(X_train, y_train)
                #Predict Output 
                predicted = model.predict(X_test)
                scoreNB = accuracy_score(y_test, predicted)
                print('Accuracy of Naive Bayes classifier on test set: {:.2f}'.format(scoreNB))
                NBFullaccuracy.append(scoreNB)
                
                # Confusion Matrix
                cfm_NB = confusion_matrix(y_test, y_pred)
                print("********************************************************")
                print("********** CONFUSION MATRIX FOR NAIVE BAYES **********")
                print("********************************************************")
                print(cfm_NB)
                
               
                # Logistic Regression for reduced dataset
                regr = LogisticRegression()
                #clf2 = regr.fit(X_trainNew, np.ravel(y_trainNew,order='C'))
                clf2 = regr.fit(X_trainNew, y_trainNew)
                y_predNew = clf2.predict(X_testNew)
                scoreLRRed = clf2.score(X_testNew, y_testNew)
                print('Accuracy of logistic regression classifier on reduced test set: {:.2f}'.format(scoreLRRed))
                LRaccuracy.append(scoreLRRed)  

                # Naive Bayes for reduced dataset
                modelRed = GaussianNB()
                # Train the model using the training sets 
                modelRed.fit(X_trainNew, y_trainNew)
                #Predict Output 
                predictedRed = modelRed.predict(X_testNew)
                scoreNBRed = accuracy_score(y_testNew, predictedRed)
                print('Accuracy of Naive Bayes classifier on reduced test set: {:.2f}'.format(scoreNBRed))
                NBaccuracy.append(scoreNBRed)
                
                
                ###############  TO BE ADDED: TEMPORAL COLUMNS FOR PREDICTION #################
                                                       
                ###############################################################################
                

                
            else:
                print('Training data contains only one class for channel: {:.0f}'.format(ch))
        
        # Plotting the graph between validation set and 
        # accuracy for both the algorithms in order to 
        # compare them. 
        if len(y_train.OccupancyClass.unique()) > 1:
            plt.scatter(FoldCount, LRaccuracy, color = 'blue')
            plt.scatter(FoldCount, NBaccuracy, color = 'red')
            plt.xlabel('Validation Set')
            plt.ylabel('Accuracy')
            plt.title('For Channel:{:.0f}'.format(ch))
            plt.show()
            
            plt.plot(FoldCount, LRaccuracy, color = 'purple')
            plt.plot(FoldCount, LRFullaccuracy, color = 'pink')
            plt.xlabel('Validation Set')
            plt.ylabel('Accuracy')
            plt.title('Logistic Regression: Full v/s Reduced dataset For Channel:{:.0f}'.format(ch))
            plt.show()

            plt.plot(FoldCount, NBaccuracy, color = 'yellow')
            plt.plot(FoldCount, NBFullaccuracy, color = 'green')
            plt.xlabel('Validation Set')
            plt.ylabel('Accuracy')
            plt.title('Naive Bayes: Full v/s Reduced dataset For Channel:{:.0f}'.format(ch))
            plt.show()

    else:
        print('Not enough data is available to run the model for channel: {:.0f}'.format(ch))
    

## Some inferences from the results:

#### 1. Reducing the columns (by removing SNR and Power_dbm) had no role in prediction. The predicted outcome was same whether or not columns were present when ran with either Logistic Regression or Naive Bayes. This implies for all our predictions we can safely ignore SNR and Power_dbm attributes. <br>

#### 2. For channels where the predicion rate was low, logistic regression performed better as compared to Naive Bayes. However, for channels where high prediction rate was achieved, both algorithm predicted similar results. This implies overall Logistic Regression performed better than Naive Bayes. </br>

#### 3. The code is ran on 7 channels- 56269, 9000, 50959, 55227, 54454, 51181, 103222. This include mix of channels from the following areas:
##### 3.a. 56269, 50959: there is only one class in training response variable (since occupancy for all occurences is 0). Such channels are skipped over in prediction.
##### 3.b. 103222: number of splits in k-fold is greater than number of samples. Such channels are also handled by skipping over them. 
##### *3.c.* 9000, 55227, 54454, 51181: these channels have enough data as well as prediction values. 